In [1]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

     |████████████████████████████████| 911kB 5.3MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [62.2 kB]
Hit:11 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:12 https://developer.download.nvidia.com/compute/cud

In [2]:
from selenium import webdriver
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
from urllib.parse import quote_plus
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import numpy as np
from datetime import datetime
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
def get_list_df(item, col, day, tag_str, playlist_name, playlist_dj, playlist_num, genre=''):
  #노래 번호 얻기
  try:
    get_item = item.find('div', {'class' : 'rank01'}).find('a')
    song_num = get_item['href'].split(',')[-1][:-2]
  except:
    print('song_num err')
    song_num = ''

  #노래제목
  try:
    song_title = get_item.text
  except:
    print('song_title err')
    song_title = ''

  #가수, 가수번호
  try:
    get_item = item.find('div', {'class' : 'rank02'}).find_all('a')
    artist_set = set()
    artist_list = []
    artist_num_list = []
    for artist_item in get_item:
      artist_tuple = (artist_item.text, artist_item['href'].split("'")[-2])
      artist_set.add(artist_tuple)
    
    for set_item in artist_set:
        artist_list.append(set_item[0])
        artist_num_list.append(set_item[1])
    artist = (' , ').join(artist_list)
    artist_num = (' ').join(artist_num_list)
  except:
    print('artist err')
    artist = ''
    artist_num=''

  #앨범이름
  try:
    get_item = item.find('div', {'class' : 'rank03'}).find('a')
    album_title = get_item.text
  except:
    print('al_title err')
    album_title = ''

  #앨범번호
  try:
    al_href = get_item['href']
    album_num = al_href.split("'")[-2]
  except:
    print('album_num err')
    album_num = ''

  #좋아요 수
  try:
    get_item = item.find('span', {'class' : 'cnt'})
    luv = get_item.text.split('\n')[-1]
  except:
    print('luv err')
    luv=''
                   #('날짜', '번호', '앨범번호', '앨범이름', '가수리스트(띄어쓰기구분)', '가수번호', '노래제목', '장르', '좋아요', '태그')
  return pd.DataFrame([day, song_num, album_num, album_title, artist, artist_num, song_title,  genre, luv, tag_str, playlist_dj, playlist_name, playlist_num, tag_str], index=col).transpose()

In [4]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)

url_main = 'https://www.melon.com/dj/today/djtoday_list.htm'
driver.get(url_main)

body = driver.find_element_by_css_selector('body')
res = driver.page_source
soup = bs(res, "html.parser")

#데이터프레임 정의
col = ['날짜', '번호', '앨범번호', '앨범이름', '가수', '가수번호', '노래제목', '장르', '좋아요', '태그', '멜론DJ', '플레이리스트', '플레이리스트_번호', '플레이리스트_태그']
df = pd.DataFrame(columns=col)

d_content = soup.find_all('ul', {'class':'d_content'})

day = datetime.today().strftime('%Y%m%d')

for content_item in d_content: 
  li_list = content_item.find_all('li')
  for li_item in li_list:

    #플레이리스트 이름 얻기
    a_info = li_item.find('a', {'class':'album_name'})
    playlist_name = a_info.text.replace('\t', '').replace('\n', '')

    try:
      #플레이리스트 제작자 이름
      a_info = li_item.find('a', {'class':'artist_name'})
      playlist_dj = a_info.text
    except:
      #DJ_멜군 이라는 dj는 span으로 생성된다.
      a_info = li_item.find('span', {'class':'artist_name'})
      playlist_dj = a_info.text

    #플레이리스트 주소 얻기
    a_info = li_item.find('a', {'class':'album_name'})
    playlist_num = a_info['href'].split("'")[-2]

    #플레이리스트 곡 수 얻기
    a_info = li_item.find('span', {'class':'tot_song'})
    playlist_song_num = a_info.text[:-1]
    
    #플레이리스트별 노래리스트로 DataFrame만들기
    playlist_url1 = 'https://www.melon.com/mymusic/dj/mymusicdjplaylistview_inform.htm?plylstSeq='
    playlist_url2 = '#params%5BplylstSeq%5D='
    playlist_url3 = '&po=pageObj&startIndex='
    pagenum = (int(playlist_song_num)-1)//50 + 1

    #page수에 맞게 반복
    for num in range(1, pagenum+1):
      #플레이리스트 노래목록 가져오기
      time.sleep(5)
      driver.get(playlist_url1 + playlist_num + playlist_url2 + playlist_num + playlist_url3 + str(num))

      playlist_body = driver.find_element_by_css_selector('body')
      playlist_res = driver.page_source
      playlist_soup = bs(playlist_res, "html.parser")
      
      #플레이리스트에 태그 이름 가져오기
      tag_div = playlist_soup.find('div', {'class':'tag_list'})
      tag_a = tag_div.find_all('a')
      tag_list = []
      for tag in tag_a:
        tag_list.append(tag.text)
      tag_str = (' ').join(tag_list)

      #노래목록 가져오기
      playlist_table_body = playlist_soup.find('tbody')
      playlist_table_item = playlist_table_body.find_all('tr')

      for list_item in playlist_table_item:
        df = pd.concat([df, get_list_df(list_item, col, day, tag_str, playlist_name, playlist_dj, playlist_num)], axis=0)

      #진행사항 확인
      print(playlist_name, num,'/', pagenum, "done")

#csv파일로 저장
df.to_csv('/content/drive/MyDrive/Colab Notebooks/세미프로젝트/일간멜론DJ/'+day+'일간멜론DJ.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


강렬한 매운맛이 느껴지는 아이돌 노동요 1 / 3 done
강렬한 매운맛이 느껴지는 아이돌 노동요 2 / 3 done
강렬한 매운맛이 느껴지는 아이돌 노동요 3 / 3 done
NEW 음악, 감각을 디자인하다. 트렌디 팝송! 1 / 1 done
여름밤에 녹아드는 낭만적인 빈티지 감성 1 / 1 done
NEW 퇴근 후 맥주 한 캔과 함께하는 감성 힙합 1 / 1 done
기분전환! 세련되고 트렌디한 사운드의 곡 1 / 2 done
기분전환! 세련되고 트렌디한 사운드의 곡 2 / 2 done
지금 당장 카페 차리고 싶어지는 달달한 R&B 1 / 1 done
7주 연속 1위의 대기록! 금주 빌보드 차트 1 / 1 done
NEW 그늘에서 듣는 시원시원한 청량미 OST 1 / 2 done
NEW 그늘에서 듣는 시원시원한 청량미 OST 2 / 2 done
더욱더 유명해져야 하는 국내 밴드 1 / 5 done
더욱더 유명해져야 하는 국내 밴드 2 / 5 done
더욱더 유명해져야 하는 국내 밴드 3 / 5 done
더욱더 유명해져야 하는 국내 밴드 4 / 5 done
더욱더 유명해져야 하는 국내 밴드 5 / 5 done
NEW 취향 가득 눌러 담은 밴드 노래 1 / 1 done
옛날 감성이 담겨있는 편안한 인디록 1 / 1 done
당신에게 꼭 들려주고픈 국내 밴드 1 / 4 done
당신에게 꼭 들려주고픈 국내 밴드 2 / 4 done
당신에게 꼭 들려주고픈 국내 밴드 3 / 4 done
당신에게 꼭 들려주고픈 국내 밴드 4 / 4 done
하우스 음악과 힙합의 만남, 힙 하우스 1 / 1 done
하우스와 레게의 퓨전, 뭄바톤 1 / 1 done
청량한 멜로디 메이커, DJ 카이고 1 / 1 done
하우스와 남미 음악의 만남, 라틴 하우스 1 / 1 done
상쾌한 리메이크 발라드 듣기 1 / 1 done
폭신한 이불 덮고 에어컨 틀기 1 / 1 done
시원하고 세련된 재즈로 힐링하기 1 / 1 done
NEW 속이 뻥 뚫리는 시원한 K-Rock 듣기 1 / 1

#테스트

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)

url_main = 'https://www.melon.com/dj/today/djtoday_list.htm'
driver.get(url_main)

body = driver.find_element_by_css_selector('body')
res = driver.page_source
soup = bs(res, "html.parser")

#데이터프레임 정의
col = ['날짜', '번호', '앨범번호', '앨범이름', '가수', '가수번호', '노래제목', '장르', '좋아요', '태그', '멜론DJ', '플레이리스트', '플레이리스트_번호', '플레이리스트_태그']
df = pd.DataFrame(columns=col)

d_content = soup.find_all('ul', {'class':'d_content'})

day = datetime.today().strftime('%Y%m%d')

content_item = d_content[0]
li_list = content_item.find_all('li')
li_item = li_list[0]

#플레이리스트 이름 얻기
a_info = li_item.find('a', {'class':'album_name'})
playlist_name = a_info.text.replace('\t', '').replace('\n', '')

try:
  #플레이리스트 제작자 이름
  a_info = li_item.find('a', {'class':'artist_name'})
  playlist_dj = a_info.text
except:
  #DJ_멜군 이라는 dj는 span으로 생성된다.
  a_info = li_item.find('span', {'class':'artist_name'})
  playlist_dj = a_info.text

#플레이리스트 주소 얻기
a_info = li_item.find('a', {'class':'album_name'})
playlist_num = a_info['href'].split("'")[-2]

#플레이리스트 곡 수 얻기
a_info = li_item.find('span', {'class':'tot_song'})
playlist_song_num = a_info.text[:-1]

#플레이리스트별 노래리스트로 DataFrame만들기
playlist_url1 = 'https://www.melon.com/mymusic/dj/mymusicdjplaylistview_inform.htm?plylstSeq='
playlist_url2 = '#params%5BplylstSeq%5D='
playlist_url3 = '&po=pageObj&startIndex='
pagenum = (int(playlist_song_num)-1)//50 + 1

#page수에 맞게 반복
for num in range(1, pagenum+1):
  #플레이리스트 노래목록 가져오기
  time.sleep(5)
  driver.get(playlist_url1 + playlist_num + playlist_url2 + playlist_num + playlist_url3 + str(num))

  playlist_body = driver.find_element_by_css_selector('body')
  playlist_res = driver.page_source
  playlist_soup = bs(playlist_res, "html.parser")
  
  #플레이리스트에 태그 이름 가져오기
  tag_div = playlist_soup.find('div', {'class':'tag_list'})
  tag_a = tag_div.find_all('a')
  tag_list = []
  for tag in tag_a:
    tag_list.append(tag.text)
  tag_str = (' ').join(tag_list)

  #노래목록 가져오기
  playlist_table_body = playlist_soup.find('tbody')
  playlist_table_item = playlist_table_body.find_all('tr')

  for list_item in playlist_table_item:
    df = pd.concat([df, get_list_df(list_item, col, day, tag_str, playlist_name, playlist_dj, playlist_num)], axis=0)

  #진행사항 확인
  print(playlist_name, num,'/', pagenum, "done")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


신나고 가슴 벅찬 비트! 일렉트로니카 1 / 2 done
신나고 가슴 벅찬 비트! 일렉트로니카 2 / 2 done
